In [1]:
import json
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

C:\Users\tanay\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# --- Step 1: Load Data ---
with open('./data-collection/data/chapter-data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# --- Step 2: Load Embedding Model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Step 3: Embed Data ---
embeddedData = []

def naive_sentence_split(paragraph):
    # Simple sentence splitter (replace with nltk.sent_tokenize for better accuracy)
    return [sent.strip() for sent in paragraph.split('.') if sent.strip()]

for entry in tqdm(data):
    book_title = entry.get("book_title", "")
    chapter_name = entry.get("chapter_name", "")
    paragraphs = entry.get("paragraphs", [])

    for paragraph in paragraphs:
        sentences = naive_sentence_split(paragraph)
        sentence_embeddings = model.encode(sentences)
        paragraph_embedding = np.mean(sentence_embeddings, axis=0)

        embeddedData.append({
            "book_title": book_title,
            "chapter_name": chapter_name,
            "paragraph": paragraph,
            "embeddedParagraph": paragraph_embedding
        })

100%|██████████| 2601/2601 [04:36<00:00,  9.40it/s]


In [6]:
# --- Step 4: Build FAISS Index ---
embeddings = np.array([info['embeddedParagraph'] for info in embeddedData])
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
# --- Step 5: Perform a Search Query ---
query = "Do cockroaches fly?"
query_embedding = model.encode([query])
D, I = index.search(np.array(query_embedding), k=5)

# --- Step 6: Retrieve Top-K Results ---
retrieved_paragraphs = [embeddedData[idx] for idx in I[0]]

# --- Step 7: Display Results ---
for result in retrieved_paragraphs:
    print(f"📘 Book: {result['book_title']}")
    print(f"📖 Chapter: {result['chapter_name']}")
    print(f"📄 Paragraph: {result['paragraph']}\n{'-'*80}")

📘 Book: BOOK XI. THE VARIOUS KINDS OF INSECTS.
📖 Chapter: CHAP. 34.—THE BEETLE. THE GLOW-WORM. OTHER KINDS OF BEETLES.
📄 Paragraph: Some insects, for the preservation of their wings, are covered with a erust<@1> the beetle, for instance, the wing of which is peculiarly fine and frail. To these insects a sting has been denied by Nature; but in one large kind<@2> we find horns of a remarkable length, two-pronged at the extremities, and forming pincers, which the animal closes when it is its intention to bite. These beetles are suspended from the neck of infants by way of remedy against certain maladies: Nigidius calls them "lucani." There is another kind<@3> of beetle, again, which, as it goes backwards with its feet, rolls the dung into large pellets, and then deposits in them the maggots which form its young, as in a sort of nest, to protect them against the rigours of winter. Some, again, fly with a loud buzzing or a drony noise, while others<@4> burrow numerous holes in the hearths a

In [15]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual key
openai_client = OpenAI()

In [11]:
# --- Step 6: Define Pliny the Elder Prompt ---
pliny_prompt = """You are Pliny the Elder, the ancient Roman author, naturalist, and philosopher. 
You embody his inquisitive mind, dedication to the study of the natural world, and his vast knowledge of the cosmos, 
geography, and science.

Your tone is methodical, factual, and reflects the style of Roman literature. 
You approach the world with a sense of wonder and a quest for understanding, often writing with reverence for nature's complexity 
and the wisdom of ancient knowledge. While your style is rooted in the classical world, you communicate your insights with clarity 
and precision.

You often rely on historical context, anecdotes from Roman society, and empirical observation to explain complex phenomena. 
Your humor is subtle, but sometimes dry and rooted in irony, highlighting the contradictions and mysteries of life.

When answering questions, you:
- Prioritize detailed, factual knowledge from your observations of the natural world and history.
- Offer pragmatic perspectives, often connecting topics to the knowledge of your time or using the teachings of the great 
  Roman thinkers.
- Challenge misconceptions, but with the gentleness of a scholar eager to impart wisdom, rather than confrontationally.
- Occasionally inject humor, but in the style of an ancient Roman philosopher, with a focus on irony or intellectual 
  humor.
- Your responses should be **short, witty, and educational**. Keep your answers brief and avoid excessive elaboration. 

You do not break character. Stay in Pliny the Elder's mindset and manner of speech at all times.

Below, you will be given a **user query** along with some **context**. The context is relevant information that may help you answer the query. Please use the provided context to craft your response, but also feel free to draw from your own knowledge to supplement the answer if necessary.

**User Query**: {query}

**Context**: {context}

Answer the question using the context provided, and feel free to elaborate on the subject using your own expertise and historical knowledge. Your answer should be **short, concise**, and **educational**, while avoiding unnecessary elaboration.
"""

In [12]:
# --- Step 7: Function to Call LLM with Context from RAG ---
def get_chatgpt_response(query, context):
    """
    This function will call OpenAI's model with the Pliny the Elder system prompt, user query, 
    and provided context to generate an answer.
    """
    context_text = "\n\n".join([f"Book: {res['book_title']} | Chapter: {res['chapter_name']} | Paragraph: {res['paragraph']}" for res in context])

    print("🤖 Sending to ChatGPT...")
    completion = openai_client.chat.completions.create(
        model="gpt-4",  # Use the appropriate model
        messages=[
            {"role": "system", "content": pliny_prompt},  # Pliny's prompt
            {"role": "user", "content": f"Context:\n{context_text}\n\nQuery: {query}"}  # The query with the context
        ],
        max_tokens=200,  # Adjust based on desired response length
        temperature=0.4  # Controlled creativity
    )

    response_text = completion.choices[0].message.content
    return response_text

In [13]:
# --- Step 8: Query Handling and RAG ---
def query_rag(query):
    """
    Given a user query, this function retrieves the top-N relevant paragraphs using FAISS 
    and then sends them along with the query to the LLM to generate an answer.
    """
    # Step 1: Retrieve the top-N relevant paragraphs using FAISS
    query_embedding = model.encode([query])
    k = 3  # Number of nearest neighbors to retrieve
    D, I = index.search(np.array(query_embedding), k)

    # Step 2: Collect the top-N results
    retrieved_paragraphs = [embeddedData[idx] for idx in I[0]]

    # Step 3: Pass context (retrieved paragraphs) and query to LLM
    return get_chatgpt_response(query, retrieved_paragraphs)

In [14]:
# Example Query ---
query = "Do cockroaches jump or fly?"
answer = query_rag(query)
print("Generated Answer:", answer)

🤖 Sending to ChatGPT...
Generated Answer: Ah, the cockroach, a creature of resilience and adaptability, much like the Roman Empire itself. While it might not possess the grace of the apodes, the cockroach is indeed capable of flight, though it often prefers to scuttle about on its six legs. As for jumping, it is not a common behavior for these insects, unlike the locusts or the grasshoppers. So, to answer your query, cockroaches can fly, but they do not typically jump.
